# Machine Learning 3: Neural networks

In [ ]:
# Import some modules that we will require
import numpy as np  
import scipy as sp
import sklearn # This is a module of machine learning tools

import matplotlib.pyplot as plt
% matplotlib inline

### Neural networks
We discussed some of the ideas and theory behind neural networks in the lectures. Here we will try them out on the classification problem that we looked at in the first notebook.


#### A Classification problem
We will now illustrate the use of a simple neural network to implement a classification of a set of moon-shaped clusters. This is an alternative approach to the one that we adopted above with a support vector machine. This time we will use a slightly more challenging data set in order to explore the issue of over-fitting. First let's load and examine the data:

In [ ]:
data = np.loadtxt('Data/moonclusterdata2.txt', delimiter=',')
classes = np.loadtxt('Data/moonclusterclasses2.txt', delimiter=',')

nclusters = 2
color = ['r','g','b']
for s in range(nclusters):
    plt.scatter(data[(classes == s),0],data[(classes == s),1],marker='o', color=color[s])
plt.xlabel('Property A')
plt.ylabel('Property B')

Notice that there is much more scatter in these clusters, which will make them harder to distinguish.

As we discussed below, in order to validate a model we will need both a set of training data and a separate set of test data, for which we know the classes (the "correct answers") but which the trained machine has never seen before. Scikit-learn contains a function to split out a set of data into two groups, as below (if you get an error, please read the note in the code cell below, but don't wory about a "DeprecationWarning"):

In [ ]:
from sklearn import cross_validation
data_train, data_test, classes_train, classes_test = sklearn.cross_validation.train_test_split(data, classes, stratify=classes, random_state=42)

# NOTE!!! If you get an error (or if you wish to eliminate the unsightly deprecation warning), then comment out the above two lines
# and uncomment the lines below, then re-execute the cell.

#from sklearn import model_selection
#data_train, data_test, classes_train, classes_test = sklearn.model_selection.train_test_split(data, classes, stratify=classes, random_state=42)



print(str(np.size(data_train,0)) + ' elements in training set, ' + str(np.size(data_test,0)) + ' elements in test set.')

We are going to use a *Multi-layer perceptron* (MLP) network, so we need to import the necessary functionality:

In [ ]:
from sklearn.neural_network import MLPClassifier

Now let's create an `mlp` object and train it using the training data:

In [ ]:
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[10,10] , activation='tanh').fit(data_train, classes_train)

The most important parameters for the MLP are the *solver*, which defines the algorithm used to train the network and the hidden layer sizes. In this case we are defining a network with two hidden layers, with 10 elements each. Finally, we also specify that the *activation function* used by each element will be a tanh function.

One way to regard a neural network is that it is a way to fit a completely general non-linear function to a set of training data. In this case, that function will define a decision boundary for classifying new data, just as for the SVM that we investigated previously. We can visualise this decision boundary as follows:

In [ ]:
X1, X2 = np.meshgrid(np.linspace(-1.5, 2.5, 1000), np.linspace(-1.0, 1.5, 1000))
X_grid = np.c_[X1.ravel(), X2.ravel()]
decision_values = mlp.predict_proba(X_grid)[:, 1]
fill_levels = [decision_values.min()] + [0] + [
            decision_values.max()]

for s in range(nclusters):
    plt.scatter(data_train[(classes_train == s),0],data_train[(classes_train == s),1],marker='o', color=color[s])
ax = plt.gca()
ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=[0.5])
plt.xlabel('Property A')
plt.ylabel('Property B')

This looks reasonable, but remember that we need to validate the trained model against the test set. So let's use the trained network to classify the test data:

In [ ]:
classes_predicted = mlp.predict(data_test)

We can evaluate the quality of the network based on what fraction of the test data are correctly classified:

In [ ]:
print('Percentage correctly classified: ' + str(np.sum(classes_test==classes_predicted)/np.size(classes_test)*100) + '%')

Not bad. In fact, we don't need to calculate this performance metric ourselves: as you might expect, scikit-learn has functions to do it for us:

In [ ]:
mlp.score(data_test,classes_test)

We can also visualise this performance by plotting the true classes of the test data along with the decision boundary:

In [ ]:
for s in range(nclusters):
    plt.scatter(data_test[(classes_test == s),0],data_test[(classes_test == s),1],marker='o', color=color[s])
ax = plt.gca()
ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=[0.5])
plt.xlabel('Property A')
plt.ylabel('Property B')

We can see that the decision boundary isn't quite right for a few of the points.

#### Overfitting
When we discussed the support vector machine, we found that if we optimised the decision boundary too finely for the training data then we ended up with poor results when we attempted to classify the test data. This is a general result and is referred to as *over-fitting*. We need to guard against it. 

In the case of a MLP neural network, we might make the mistake of using too large a number of neurons in the hidden layers. This would give a very detailed fit to the training data. Let's try a network with two layers of 100 elements each and plot the resulting boundary against the test data (note that this might take a little while to complete):

In [ ]:
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[100,100] , activation='tanh').fit(data_train, classes_train)
classes_predicted = mlp.predict(data_test)
print('Score: ', mlp.score(data_test,classes_test))

X1, X2 = np.meshgrid(np.linspace(-1.5, 2.5, 1000), np.linspace(-1.0, 1.5, 1000))
X_grid = np.c_[X1.ravel(), X2.ravel()]
decision_values = mlp.predict_proba(X_grid)[:, 1]
fill_levels = [decision_values.min()] + [0] + [
            decision_values.max()]

for s in range(nclusters):
    plt.scatter(data_test[(classes_test == s),0],data_test[(classes_test == s),1],marker='o', color=color[s])
ax = plt.gca()
ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=[0.5])
plt.xlabel('Property A')
plt.ylabel('Property B')

Our more sophisticated (larger) network has a *worse* performance on the test set. One way to guard against over-fitting is to systematically explore a range of values for the parameters of the model that you are using (in this case the size of the hidden layers) and plot the performance of the network against both the training and the test set as a function of the parameters. Below we do this for the case of the two layer MLP, using a `for` loop:

In [ ]:
nunits = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20]
score_train=[]
score_test=[]
for s in nunits:
    mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[s,s], activation='tanh').fit(data_train, classes_train)
    score_train.append(mlp.score(data_train,classes_train))
    score_test.append(mlp.score(data_test,classes_test))
plt.plot(nunits,score_train,'ro-', label="Training data")
plt.plot(nunits,score_test,'go-', label="Test data")
plt.legend()
plt.xlabel('Number of elements per layer')
plt.ylabel('Performance score')

What do we see? First, as the size of the network increases, the performance against the training set gradually increases. This makes sense, because more elements in the hidden layers will give a decision boundary that more finely fits the details of the training set. However, we see that there is an optimum value for the network size against the test set. Once we get above 4 elements per hidden layer the model starts to over-fit the training data and performance against the test data plateaus and even begins to degrade.

#### <span style="color: red"> Task 5:</span>   Copy the above code and use it to explore the performance of a *single-layer* MLP as a function of network size. (hint: you will need to specify the network size with a single number as follows:   `hidden_layer_sizes=[s]`.)

You might find that the optimised single-layer network performs slightly better than the two-layer network. This is probably just luck, as we can see if we show the decision boundaries from the optimised single and dual layer networks together. The performance is very sensitive to the details of the shape of the boundary in a small region around (0.0,0.5) in property space:

In [ ]:
X1, X2 = np.meshgrid(np.linspace(-1.5, 2.5, 1000), np.linspace(-1.0, 1.5, 1000))
X_grid = np.c_[X1.ravel(), X2.ravel()]

mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[4] , activation='tanh').fit(data_train, classes_train)
decision_values_single = mlp.predict_proba(X_grid)[:, 1]
fill_levels_single = [decision_values.min()] + [0] + [
            decision_values.max()]

mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[4,4] , activation='tanh').fit(data_train, classes_train)
decision_values_double = mlp.predict_proba(X_grid)[:, 1]
fill_levels_double = [decision_values.min()] + [0] + [
            decision_values.max()]

for s in range(nclusters):
    plt.scatter(data_test[(classes_test == s),0],data_test[(classes_test == s),1],marker='o', color=color[s])
ax = plt.gca()
ax.contour(X1, X2, decision_values_single.reshape(X1.shape), levels=[0.5], colors=('b'))
ax.contour(X1, X2, decision_values_double.reshape(X1.shape), levels=[0.5], colors=('k'))
plt.xlabel('Property A')
plt.ylabel('Property B')

We hope you have enjoyed exploring some of the concepts behind machine learning and gained some appreciation of the types of methods that can be used and the types of problems that can be tackled. Also, note how easy python makes all this, with the help of scikit-learn.